In [2]:
from PIL import Image
from pathlib import Path
import argparse
import glob
import os
import pandas as pd
from re import search
import re


In [4]:
path = '/home/ubuntu/bucket/projects'
projdir = '2019_07_11_JUMP-CP-pilots'
batchname = '2020_10_27_Scope1_YokogawaJapan'
batchpath=os.path.join(path, projdir,batchname)


In [7]:
direc, sub, files = os.walk(os.path.join(path, projdir, batchname))


ValueError: too many values to unpack (expected 3)

In [6]:
plist = [f for f in os.listdir(os.path.join(path, projdir, batchname))]

plist

['images']

In [3]:
plist = [f for f in os.listdir(os.path.join(path, projdir, batchname))]

platelist = []


outpath_list = []


for p in plist:


    pl = p.split("-")
    
    plate = '_'.join(pl)
    
    
    outpath = os.path.join(path, projdir, 'workspace', 'load_data_csv', batchname, plate)
    
    
    
#     def makedirectory():

#         try:
#             os.makedirs(outpath)

#             print("Directory", outpath, "is created")

#         except IOError:

#             print("Directory", outpath, "already exists")

#     directory = makedirectory()
    
    
    platelist.append(plate)
    
    outpath_list.append(outpath)
        
        


In [4]:
plist_path = [os.path.join(path, projdir, batchname, f, 'PECCU') for f in os.listdir(os.path.join(path, projdir, batchname))]

plist_path

['/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_16_Scope1_YokogawaUS/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU',
 '/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_16_Scope1_YokogawaUS/BRO0117014-3D-2x2x1-singleplane-20x_20201026_113115/PECCU',
 '/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_16_Scope1_YokogawaUS/BRO0117033-3D-2x2x1-dpc-20x_20201023_151718/PECCU',
 '/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_16_Scope1_YokogawaUS/BRO0117033-3D-2x2x1-singleplane-20x_20201015_161709/PECCU',
 '/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_16_Scope1_YokogawaUS/BRO0117056-2x2x1-20x_20201022_110522/PECCU',
 '/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_16_Scope1_YokogawaUS/BRO0117056-3D-2x2x1-20x_20201022_173956/PECCU',
 '/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_16_Scope1_YokogawaUS/BRO0117059-40x_20201013_155609/PECCU',
 '/home/ubuntu/bucket/projects/201

In [10]:
## Channel mapping

# channels = {'ch1':"OrigDNA",
#     'ch2':"OrigER",
#     'ch3':"OrigActin_Golgi",
#     'ch4':"OrigMito",
#     'ch5':"OrigNucleoli",
#     'ch6':"OrigBrightField",
#     }

channels = {'ch1':"OrigDNA",
    'ch2':"OrigER",
    'ch3':"OrigActin_Golgi",
    'ch4':"OrigMito",
    'ch5':"OrigNucleoli",
    }

In [11]:
fpath = plist_path[3]

outpath = outpath_list[3]



plate = os.path.split(outpath)[-1]

print(fpath)
print(outpath)
print(plate)

/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_16_Scope1_YokogawaUS/BRO0117033-3D-2x2x1-singleplane-20x_20201015_161709/PECCU
/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/workspace/load_data_csv/2020_11_16_Scope1_YokogawaUS/BRO0117033_3D_2x2x1_singleplane_20x_20201015_161709
BRO0117033_3D_2x2x1_singleplane_20x_20201015_161709


In [15]:
def load_data(channels):
    
    lst = []

    for i, image in enumerate(os.listdir(fpath)):
    
        imgpath = os.path.join(fpath, image)


        if imgpath.endswith("tif"):

                head, tail = os.path.split(imgpath)
                
                if "PECCU" in tail:
            
        
                    lst.append(tail)


        keys = list(channels.keys())
        values = list(channels.values())


        ch1 = [s for s in lst if 'C01.' in s]
        ch2 = [s for s in lst if 'C02.' in s]
        ch3 = [s for s in lst if 'C03.' in s]
        ch4 = [s for s in lst if 'C04.' in s]
        ch5 = [s for s in lst if 'C05.' in s]

        
    

        zippedlist = list(zip(ch1, ch2, ch3, ch4, ch5))

        df = pd.DataFrame(zippedlist, columns=["FileName_"+chname for chname in values])

        path_columns = { c : fpath for c in ["PathName_"+chname for chname in values]}

        df = df.assign(**path_columns)


        wellname = df['FileName_OrigDNA'].tolist()



        pattern = re.compile(".+_(?P<Well>\w+)_.+F0(?P<site>\d+)")



        match = [pattern.match(i) for i in wellname]
        well = [r.group("Well") for r in match]
        site= [f.group("site") for f in match]

        site_0= [f.group("site") for f in match]

        site = [(lambda x: x.strip('0') if isinstance(x,str) and len(x) != 1 else x)(x) for x in site_0]



        df['Metadata_Well'] = well
        df['Metadata_Site'] = site
        df['Metadata_Plate'] = plate



        colnames = ['FileName_'+values[0], 'PathName_'+values[0],
                    'FileName_'+values[1], 'PathName_'+values[1],
                    'FileName_'+values[2], 'PathName_'+values[2],
                    'FileName_'+values[3], 'PathName_'+values[3],
                    'FileName_'+values[4], 'PathName_'+values[4],
                    'Metadata_Plate','Metadata_Well', 
                    'Metadata_Site']




        
   
    
    return df.reindex(columns=colnames)

df = load_data(channels) 

df.to_csv(outpath + "/" + 'load_data.csv', index=False)

In [16]:
def load_data_with_illum(channels):
    
    illum_channels = list(channels.values())
    


    illum = [s.replace("Orig", "Illum") for s in illum_channels]

    df['FileName_'+illum[0]] = plate + '_' + illum[0] + '.npy'

    df['PathName_'+illum[0]] = os.path.join(batchpath, 'illum', plate)


    df['FileName_'+illum[1]] = plate + '_' + illum[1] + '.npy'

    df['PathName_'+illum[1]] = os.path.join(batchpath, 'illum', plate)


    df['FileName_'+illum[2]] = plate + '_' + illum[2] + '.npy'

    df['PathName_'+illum[2]] = os.path.join(batchpath, 'illum', plate)



    df['FileName_'+illum[3]] = plate + '_' + illum[3] + '.npy'

    df['PathName_'+illum[3]] = os.path.join(batchpath, 'illum', plate)


    df['FileName_'+illum[4]] = plate + '_' + illum[4] + '.npy'

    df['PathName_'+illum[4]] = os.path.join(batchpath, 'illum', plate)

    


       
    return df.reindex()

illum = load_data_with_illum(channels)

illum.to_csv(outpath + "/" + 'load_data_with_illum.csv', index=False)